## YOLO Parking Spot Detection 🚗🅿️


###  Setup Environment

In [17]:
import sys
import os
import shutil
import torch

# Add the path to the utils folder to the system path
if os.path.basename(os.getcwd()) == 'notebooks':
    sys.path.append('../')

from utils.functions import is_custom_model

repo_path = os.getcwd()
repo_path = os.path.dirname(repo_path)

yolov8_path = os.path.join(repo_path, 'yolov8')

# Check if GPU is available and print device information
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Current GPU memory allocated: {torch.cuda.memory_allocated(0)} bytes")
    print(f"Current GPU memory cached: {torch.cuda.memory_reserved(0)} bytes")
else:
    device = torch.device('cpu')
    print("No GPU available, using CPU.")

# You can also print the current device in use
print(f"Device in use: {device}")


No GPU available, using CPU.
Device in use: cpu


### Install YOLOv8 and Prepare Dataset

In [18]:
# Install yolov8
%pip install ultralytics

# Create yolov8 custom data
!python $repo_path/utils/yolov8start.py --reporoot $repo_path


Note: you may need to restart the kernel to use updated packages.


Traceback (most recent call last):
  File "g:\Project-repo\utils\yolov8start.py", line 78, in <module>
    main(opt)
  File "g:\Project-repo\utils\yolov8start.py", line 53, in main
    shutil.rmtree(yolo_path)
  File "c:\Users\Asiri\AppData\Local\Programs\Python\Python310\lib\shutil.py", line 739, in rmtree
    return _rmtree_unsafe(path, onerror)
  File "c:\Users\Asiri\AppData\Local\Programs\Python\Python310\lib\shutil.py", line 621, in _rmtree_unsafe
    onerror(os.rmdir, path, sys.exc_info())
  File "c:\Users\Asiri\AppData\Local\Programs\Python\Python310\lib\shutil.py", line 619, in _rmtree_unsafe
    os.rmdir(path)
PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'g:\\Project-repo\\yolov8'


In [19]:
%cd $yolov8_path

g:\Project-repo\yolov8


c:\Users\Asiri\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [35]:
def yolov8_detect_predict(model, device="cpu", conf=0.25, iou=0.45):
    # Check if the custom model is available
    model, model_path = is_custom_model(model, "8")

    # Debug information to ensure the model and paths are correct
    print(f"Model: {model}")
    print(f"Model path: {model_path}")
    print(f"Device: {device}")
    print(f"Confidence Threshold: {conf}")
    print(f"IOU Threshold: {iou}")
    
    # Ensure the model path exists
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model path does not exist: {model_path}")

    # Ensure the source directory exists
    source_path = os.path.join(repo_path, "data/images")
    if not os.path.exists(source_path):
        raise FileNotFoundError(f"Source path does not exist: {source_path}")

    # Remove existing results if present
    results_path = os.path.join(repo_path, f'results/{model[:-3]}/')
    if os.path.exists(results_path):
        shutil.rmtree(results_path)
    
    # Debug: Print paths to confirm correct inputs
    print(f"Source path: {source_path}")
    print(f"Results path: {results_path}")

    # Execute the YOLOv8 predict command
    try:
        !yolo task=detect mode=predict \
            model="{model_path}" \
            source="{source_path}" \
            save_txt=True \
            exist_ok=True \
            imgsz=640 \
            conf={conf} \
            iou={iou} \
            project="{repo_path}/results/" \
            name="{model[:-3]}/yolo_images" \
            device="{device}"
        
        # After prediction, move labels to the correct folder
        labels_source = os.path.join(repo_path, f'results/{model[:-3]}/yolo_images/labels')
        labels_dest = os.path.join(repo_path, f'results/{model[:-3]}/labels')

        # Check if the labels were generated
        if os.path.exists(labels_source):
            shutil.move(labels_source, labels_dest)
            print(f"Labels moved to: {labels_dest}")
        else:
            print("Warning: Labels directory not found after prediction.")
    
    except Exception as e:
        print(f"Error during YOLOv8 prediction: {e}")

    

In [36]:
yolov8_detect_predict(f'{repo_path}/models/parking-lot-prediction_ultralytics-100e-2.pt')

Model: parking-lot-prediction_ultralytics-100e-2.pt
Model path: g:\Project-repo/models/parking-lot-prediction_ultralytics-100e-2.pt
Device: cpu
Confidence Threshold: 0.25
IOU Threshold: 0.45
Source path: g:\Project-repo\data/images
Results path: g:\Project-repo\results/parking-lot-prediction_ultralytics-100e-2/
